In [1]:
# Import the necessary libraries
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)  # To show all rows
pd.set_option('display.max_columns', None)  # To show all columns

In [2]:
# Read in the CSV files
googl_df = pd.read_csv("Resources/googl_df.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)
nvda_df = pd.read_csv("Resources/nvda_df.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)
mmm_df = pd.read_csv("Resources/mmm_df.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)
pg_df = pd.read_csv("Resources/pg_df.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)

In [3]:
# Slice the "GOOGL Adj. Close" column from the DataFrame to create the "signals_googl_df" DataFrame
signals_googl_df = googl_df.loc[:,['GOOGL Adj. Close']]
signals_nvda_df = nvda_df.loc[:,['NVDA Adj. Close']]
signals_mmm_df = mmm_df.loc[:,['MMM Adj. Close']]
signals_pg_df = pg_df.loc[:,['PG Adj. Close']]

# Create lists of signal and ticker DataFrames for iteration
signal_dfs = [signals_googl_df, signals_nvda_df, signals_mmm_df, signals_pg_df]
ticker_dfs = [googl_df, nvda_df, mmm_df, pg_df]
df_pairs = [(signals_googl_df, googl_df), (signals_nvda_df, nvda_df), (signals_mmm_df, mmm_df), (signals_pg_df, pg_df)]

# Create a list of individual stock DataFrame names and columns to feed loops
ticker_data = [(signals_googl_df, "GOOGL Adj. Close"),(signals_nvda_df, "NVDA Adj. Close"),(signals_mmm_df, "MMM Adj. Close"),(signals_pg_df, "PG Adj. Close")]

In [4]:
# Create and Populate the "signal" column for signal DataFrames
for df in signal_dfs:
    df['Signal'] = 0.0
    df['Signal_ema'] = 0.0
    df['Signal_bb_rsi'] = 0.0

In [5]:
# Generate the trading signal 0 or 1,
# where 1 is when sma_fast is greater than sma_slow or ema_fast is greater than ema_slow
# and 0 otherwise

for signals_df, source_df in df_pairs:
    # Apply the condition to set the 'Signal' column
    signals_df['Signal'] = np.where(source_df['sma_fast'] > source_df['sma_slow'], 1.0, 0.0)
    signals_df['Signal_ema'] = np.where(source_df['ema_fast'] > source_df['ema_slow'], 1.0, 0.0)

In [6]:
# Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
for signals_df, source_df in df_pairs:
    signals_df['Entry/Exit'] = signals_df['Signal'].diff()
    signals_df['Entry/Exit_ema'] = signals_df['Signal_ema'].diff()

### Bollinger Band and RSI Trading Signals and Entry/Exit points

In [7]:
# Generate the trading signals 0 or 1
signals_googl_df['bb_rsi_buy'] = np.where((signals_googl_df['GOOGL Adj. Close'] < googl_df['bb_lower_talib']) & (googl_df['RSI'] < 30), 1, 0)
signals_googl_df['bb_rsi_sell'] = np.where((signals_googl_df['GOOGL Adj. Close'] > googl_df['bb_upper_talib']) & (googl_df['RSI'] > 70), 1, 0)

signals_nvda_df['bb_rsi_buy'] = np.where((signals_nvda_df['NVDA Adj. Close'] < nvda_df['bb_lower_talib']) & (nvda_df['RSI'] < 30), 1, 0)
signals_nvda_df['bb_rsi_sell'] = np.where((signals_nvda_df['NVDA Adj. Close'] > nvda_df['bb_upper_talib']) & (nvda_df['RSI'] > 70), 1, 0)

signals_mmm_df['bb_rsi_buy'] = np.where((signals_mmm_df['MMM Adj. Close'] < mmm_df['bb_lower_talib']) & (mmm_df['RSI'] < 30), 1, 0)
signals_mmm_df['bb_rsi_sell'] = np.where((signals_mmm_df['MMM Adj. Close'] > mmm_df['bb_upper_talib']) & (mmm_df['RSI'] > 70), 1, 0)

signals_pg_df['bb_rsi_buy'] = np.where((signals_pg_df['PG Adj. Close'] < pg_df['bb_lower_talib']) & (pg_df['RSI'] < 30), 1, 0)
signals_pg_df['bb_rsi_sell'] = np.where((signals_pg_df['PG Adj. Close'] > pg_df['bb_upper_talib']) & (pg_df['RSI'] > 70), 1, 0)

In [8]:
# Convert sell signals to -1
for df in signal_dfs:
    df['bb_rsi_sell'] *= -1
    # Combine bb buy and sell signals into 'Signal_bb_rsi' column
    df['Signal_bb_rsi'] = df['bb_rsi_buy'] + df['bb_rsi_sell']
    #print(df)

signals_googl_df[bb_rsi_sell'] *= -1
signals_googl_df['Signal_bb_rsi'] = signals_googl_df['bb_rsi_buy'] + signals_googl_df['bb_rsi_sell']

signals_nvda_df[bb_rsi_sell'] *= -1
signals_nvda_df['Signal_bb_rsi'] = signals_nvda_df['bb_rsi_buy'] + signals_nvda_df['bb_rsi_sell']



            GOOGL Adj. Close  Signal  Signal_ema  Signal_bb_rsi  Entry/Exit  \
Date                                                                          
2019-01-02          52.73400     0.0         0.0              0         NaN   
2019-01-03          51.27350     0.0         0.0              0         0.0   
2019-01-04          53.90350     0.0         1.0              0         0.0   
2019-01-07          53.79600     0.0         1.0              0         0.0   
2019-01-08          54.26850     0.0         1.0              0         0.0   
2019-01-09          54.08250     0.0         1.0              0         0.0   
2019-01-10          53.94150     0.0         1.0              0         0.0   
2019-01-11          53.22350     0.0         1.0              0         0.0   
2019-01-14          52.57550     0.0         1.0              0         0.0   
2019-01-15          54.32550     0.0         1.0              0         0.0   
2019-01-16          54.47550     0.0         1.0    

In [10]:
# Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
for signals_df, source_df in df_pairs:
    signals_df['Entry/Exit_bb_rsi'] = signals_df['Signal_bb_rsi'].diff()
    #signals_df['Signal_bb_rsi'] = signals_df['Signal_bb_rsi'].fillna(0, inplace=True)
    #signals_df['Entry/Exit_bb_rsi'] = signals_df['Signal_bb_rsi']    

In [11]:
# Drop unnecessary calculation columns
for df in signal_dfs:
    df.drop(columns=['bb_rsi_buy', 'bb_rsi_sell'], inplace=True)

### SMA Entry/Exit Plots

In [27]:
signals_googl_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1320 entries, 2019-01-02 to 2024-04-01
Data columns (total 28 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   GOOGL Adj. Close                     1320 non-null   float64
 1   Signal                               1320 non-null   float64
 2   Signal_ema                           1320 non-null   float64
 3   Signal_bb_rsi                        1320 non-null   int64  
 4   Entry/Exit                           1319 non-null   float64
 5   Entry/Exit_ema                       1319 non-null   float64
 6   Entry/Exit_bb_rsi                    1319 non-null   float64
 7   Position                             1320 non-null   float64
 8   Position_ema                         1320 non-null   float64
 9   Position_bb_rsi                      1320 non-null   int64  
 10  Entry/Exit Position                  1319 non-null   float64
 11  Entry/Exit P

In [12]:
# Initialize a dictionary to store the plots for SMA strategy for each stock
plots_sma = {}

for signals_df, source_df in df_pairs:
    # Extract the stock symbol from the column names (assuming 'Adj. Close' is part of the name)
    stock_symbol = [col for col in signals_df.columns if 'Adj. Close' in col][0].split()[0]

    # Plot exit positions
    exit = signals_df[signals_df['Entry/Exit'] == -1.0][f'{stock_symbol} Adj. Close'].hvplot.scatter(
        color='orange',
        marker='v',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Plot entry positions
    entry = signals_df[signals_df['Entry/Exit'] == 1.0][f'{stock_symbol} Adj. Close'].hvplot.scatter(
        color='purple',
        marker='^',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Plot security close price
    security_close = signals_df[[f'{stock_symbol} Adj. Close']].hvplot(
        line_color='lightgray',
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Plot moving averages
    moving_avgs = source_df[['sma_fast', 'sma_slow']].hvplot(
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Create the overlay plot
    entry_exit_plot = security_close * moving_avgs * entry * exit

    # Set the title dynamically based on the stock symbol
    entry_exit_plot = entry_exit_plot.opts(
        title=f"{stock_symbol} - SMA10, SMA100, Entry and Exit Points"
    )

    # Store the plot in the dictionary using the stock symbol as key
    plots_sma[stock_symbol] = entry_exit_plot

### EMA Entry/Exit Plots

In [13]:
# Initialize a dictionary to store the plots for EMA strategy for each stock
plots_ema = {}

for signals_df, source_df in df_pairs:
    # Extract the stock symbol from the column names (assuming 'Adj. Close' is part of the name)
    stock_symbol = [col for col in signals_df.columns if 'Adj. Close' in col][0].split()[0]

    # Plot exit positions
    exit = signals_df[signals_df['Entry/Exit_ema'] == -1.0][f'{stock_symbol} Adj. Close'].hvplot.scatter(
        color='orange',
        marker='v',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Plot entry positions
    entry = signals_df[signals_df['Entry/Exit_ema'] == 1.0][f'{stock_symbol} Adj. Close'].hvplot.scatter(
        color='purple',
        marker='^',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Plot security close price
    security_close = signals_df[[f'{stock_symbol} Adj. Close']].hvplot(
        line_color='lightgray',
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Plot moving averages
    moving_avgs = source_df[['ema_fast', 'ema_slow']].hvplot(
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Create the overlay plot
    entry_exit_plot = security_close * moving_avgs * entry * exit

    # Set the title dynamically based on the stock symbol
    entry_exit_plot = entry_exit_plot.opts(
        title=f"{stock_symbol} - EMA10, EMA100, Entry and Exit Points"
    )

    # Store the plot in the dictionary using the stock symbol as key
    plots_ema[stock_symbol] = entry_exit_plot

### BB and RSI Entry/Exit Plots

In [14]:
# Initialize a dictionary to store the plots for BB & RSI strategy for each stock
plots_bb_rsi = {}

for signals_df, source_df in df_pairs:
    # Extract the stock symbol from the column names (assuming 'Adj. Close' is part of the name)
    stock_symbol = [col for col in signals_df.columns if 'Adj. Close' in col][0].split()[0]

    # Plot exit positions
    exit = signals_df[signals_df['Entry/Exit_bb_rsi'] == -1.0][f'{stock_symbol} Adj. Close'].hvplot.scatter(
        color='orange',
        marker='v',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Plot entry positions
    entry = signals_df[signals_df['Entry/Exit_bb_rsi'] == 1.0][f'{stock_symbol} Adj. Close'].hvplot.scatter(
        color='purple',
        marker='^',
        size=200,
        legend=False,
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Plot security close price
    security_close = signals_df[[f'{stock_symbol} Adj. Close']].hvplot(
        line_color='lightgray',
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Plot moving averages
    moving_avgs = source_df[['bb_upper_talib', 'bb_mid_talib', 'bb_lower_talib', 'RSI']].hvplot(
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Create the overlay plot
    entry_exit_plot = security_close * moving_avgs * entry * exit

    # Set the title dynamically based on the stock symbol
    entry_exit_plot = entry_exit_plot.opts(
        title=f"{stock_symbol} - Bollinger Bands & RSI, Entry and Exit Points"
    )

    # Store the plot in the dictionary using the stock symbol as key
    plots_bb_rsi[stock_symbol] = entry_exit_plot

### Investment Capital Tracking

In [15]:
# Set initial investment capital
initial_capital = float(100000)

# Set the share size per transaction
share_size = 500

In [16]:
# Creating a position column to store the number of shares held
# Buy a 500 share position when the dual moving average crossover Signal equals 1
# Otherwise, `Position` should be zero (sell)

for df in signal_dfs:
    df['Position'] = share_size * df['Signal']
    df['Position_ema'] = share_size * df['Signal_ema']
    df['Position_bb_rsi'] = share_size * df['Signal_bb_rsi']

In [17]:
# Determine the points in time where the share position is bought or sold

for df in signal_dfs:
    df['Entry/Exit Position'] = df['Position'].diff()
    df['Entry/Exit Position_ema'] = df['Position_ema'].diff()
    df['Entry/Exit Position_bb_rsi'] = df['Position_bb_rsi'].diff()

In [18]:
# Multiply the close price by the number of shares held, or the Position

for df, adj_close_col in ticker_data:
    df['Portfolio Holdings'] = df[adj_close_col] * df['Position']
    df['Portfolio Holdings_ema'] = df[adj_close_col] * df['Position_ema']
    df['Portfolio Holdings_bb_rsi'] = df[adj_close_col] * df['Position_bb_rsi']

In [19]:
# Subtract the amount of either the cost or proceeds of the trade from the initial capital invested

for df, adj_close_col in ticker_data:
    df['Portfolio Cash'] = initial_capital - (df[adj_close_col] * df['Entry/Exit Position']).cumsum()
    df['Portfolio Cash_ema'] = initial_capital - (df[adj_close_col] * df['Entry/Exit Position_ema']).cumsum()
    df['Portfolio Cash_bb_rsi'] = initial_capital - (df[adj_close_col] * df['Entry/Exit Position_bb_rsi']).cumsum()

In [20]:
# Calculate the total portfolio value by adding the portfolio cash to the portfolio holdings (or investments)

for df in signal_dfs:
    df['Portfolio Total'] = df['Portfolio Cash'] + df['Portfolio Holdings']
    df['Portfolio Total_ema'] = df['Portfolio Cash_ema'] + df['Portfolio Holdings_ema']
    df['Portfolio Total_bb_rsi'] = df['Portfolio Cash_bb_rsi'] + df['Portfolio Holdings_bb_rsi']

In [21]:
# Calculate the portfolio daily returns

for df in signal_dfs:
    df['Portfolio Daily Returns'] = df['Portfolio Total'].pct_change()
    df['Portfolio Daily Returns_ema'] = df['Portfolio Total_ema'].pct_change()
    df['Portfolio Daily Returns_bb_rsi'] = df['Portfolio Total_bb_rsi'].pct_change()

In [22]:
# Calculate the portfolio cumulative returns

for df in signal_dfs:
    df['Portfolio Cumulative Returns'] = (1 + df['Portfolio Daily Returns']).cumprod() - 1
    df['Portfolio Cumulative Returns_ema'] = (1 + df['Portfolio Daily Returns_ema']).cumprod() - 1
    df['Portfolio Cumulative Returns_bb_rsi'] = (1 + df['Portfolio Daily Returns_bb_rsi']).cumprod() - 1

### SMA Portfolio Value Plots for each ticker

In [23]:
# Initialize a dictionary to store the plots
plots_sma_portfolio_value = {}

for df in signal_dfs:
    
    # Extract the stock symbol from the column names (assuming 'Adj. Close' is part of the name)
    stock_symbol = [col for col in df.columns if 'Adj. Close' in col][0].split()[0]

    # Visualize exit position relative to total portfolio value
    exit = df[df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
        color='orange',
        marker='v',
        size=200,
        legend=False,
        ylabel='Total Portfolio Value',
        width=1000,
        height=400
    )

    # Visualize entry position relative to total portfolio value
    entry = df[df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
        color='purple',
        marker='^',
        size=200,
        ylabel='Total Portfolio Value',
        width=1000,
        height=400
    )

    # Visualize the value of the total portfolio
    total_portfolio_value = df[['Portfolio Total']].hvplot(
        line_color='lightgray',
        ylabel='Total Portfolio Value',
        xlabel='Date',
        width=1000,
        height=400
    )

    # Overlay the plots
    portfolio_entry_exit_plot = total_portfolio_value * entry * exit

    # Set the title dynamically based on the stock symbol
    portfolio_entry_exit_plot = portfolio_entry_exit_plot.opts(
        title=f"{stock_symbol} - SMA10, SMA100, Total Portfolio Value",
        yformatter='%.0f'
    )

    # Store the plot in the dictionary using the stock symbol as key
    plots_sma_portfolio_value[stock_symbol] = portfolio_entry_exit_plot

### EMA Portfolio Value Plots for each ticker

In [24]:
# Initialize a dictionary to store the plots
plots_ema_portfolio_value = {}

for df in signal_dfs:
    
    # Extract the stock symbol from the column names (assuming 'Adj. Close' is part of the name)
    stock_symbol = [col for col in df.columns if 'Adj. Close' in col][0].split()[0]

    # Visualize exit position relative to total portfolio value
    exit = df[df['Entry/Exit_ema'] == -1.0]['Portfolio Total_ema'].hvplot.scatter(
        color='orange',
        marker='v',
        size=200,
        legend=False,
        ylabel='Total Portfolio Value',
        width=1000,
        height=400
    )

    # Visualize entry position relative to total portfolio value
    entry = df[df['Entry/Exit_ema'] == 1.0]['Portfolio Total_ema'].hvplot.scatter(
        color='purple',
        marker='^',
        size=200,
        ylabel='Total Portfolio Value',
        width=1000,
        height=400
    )

    # Visualize the value of the total portfolio
    total_portfolio_value = df[['Portfolio Total_ema']].hvplot(
        line_color='lightgray',
        ylabel='Total Portfolio Value',
        xlabel='Date',
        width=1000,
        height=400
    )

    # Overlay the plots
    portfolio_entry_exit_plot = total_portfolio_value * entry * exit

    # Set the title dynamically based on the stock symbol
    portfolio_entry_exit_plot = portfolio_entry_exit_plot.opts(
        title=f"{stock_symbol} - EMA10, EMA100, Total Portfolio Value",
        yformatter='%.0f'
    )

    # Store the plot in the dictionary using the stock symbol as key
    plots_ema_portfolio_value[stock_symbol] = portfolio_entry_exit_plot

### BB & RSI Portfolio Value Plots for each ticker

In [25]:
# Initialize a dictionary to store the plots
plots_bb_rsi_portfolio_value = {}

for df in signal_dfs:
    
    # Extract the stock symbol from the column names (assuming 'Adj. Close' is part of the name)
    stock_symbol = [col for col in df.columns if 'Adj. Close' in col][0].split()[0]

    # Visualize exit position relative to total portfolio value
    exit = df[df['Entry/Exit_bb_rsi'] == -1.0]['Portfolio Total_bb_rsi'].hvplot.scatter(
        color='orange',
        marker='v',
        size=200,
        legend=False,
        ylabel='Total Portfolio Value',
        width=1000,
        height=400
    )

    # Visualize entry position relative to total portfolio value
    entry = df[df['Entry/Exit_bb_rsi'] == 1.0]['Portfolio Total_bb_rsi'].hvplot.scatter(
        color='purple',
        marker='^',
        size=200,
        ylabel='Total Portfolio Value',
        width=1000,
        height=400
    )

    # Visualize the value of the total portfolio
    total_portfolio_value = df[['Portfolio Total_bb_rsi']].hvplot(
        line_color='lightgray',
        ylabel='Total Portfolio Value',
        xlabel='Date',
        width=1000,
        height=400
    )

    # Overlay the plots
    portfolio_entry_exit_plot = total_portfolio_value * entry * exit

    # Set the title dynamically based on the stock symbol
    portfolio_entry_exit_plot = portfolio_entry_exit_plot.opts(
        title=f"{stock_symbol} - Bollinger Bands & RSI Total Portfolio Value",
        yformatter='%.0f'
    )

    # Store the plot in the dictionary using the stock symbol as key
    plots_bb_rsi_portfolio_value[stock_symbol] = portfolio_entry_exit_plot

### Display All Plots

In [26]:
# Display the SMA plots
display(plots_sma['GOOGL'])
display(plots_ema['GOOGL'])
display(plots_bb_rsi['GOOGL'])
display(plots_sma_portfolio_value['GOOGL'])
display(plots_ema_portfolio_value['GOOGL'])
display(plots_bb_rsi_portfolio_value['GOOGL'])


display(plots_sma['NVDA'])
display(plots_ema['NVDA'])
display(plots_bb_rsi['NVDA'])
display(plots_sma_portfolio_value['NVDA'])
display(plots_ema_portfolio_value['NVDA'])
display(plots_bb_rsi_portfolio_value['NVDA'])

display(plots_sma['MMM'])
display(plots_ema['MMM'])
display(plots_bb_rsi['MMM'])
display(plots_sma_portfolio_value['MMM'])
display(plots_ema_portfolio_value['MMM'])
display(plots_bb_rsi_portfolio_value['MMM'])

display(plots_sma['PG'])
display(plots_ema['PG'])
display(plots_bb_rsi['PG'])
display(plots_sma_portfolio_value['PG'])
display(plots_ema_portfolio_value['PG'])
display(plots_bb_rsi_portfolio_value['PG'])

:Overlay
   .Curve.I                              :Curve   [Date]   (GOOGL Adj. Close)
   .NdOverlay.I                          :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.GOOGL_Adj_full_stop_Close.I  :Scatter   [Date]   (GOOGL Adj. Close)
   .Scatter.GOOGL_Adj_full_stop_Close.II :Scatter   [Date]   (GOOGL Adj. Close)

:Overlay
   .Curve.I                              :Curve   [Date]   (GOOGL Adj. Close)
   .NdOverlay.I                          :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.GOOGL_Adj_full_stop_Close.I  :Scatter   [Date]   (GOOGL Adj. Close)
   .Scatter.GOOGL_Adj_full_stop_Close.II :Scatter   [Date]   (GOOGL Adj. Close)

:Overlay
   .Curve.I                              :Curve   [Date]   (GOOGL Adj. Close)
   .NdOverlay.I                          :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.GOOGL_Adj_full_stop_Close.I  :Scatter   [Date]   (GOOGL Adj. Close)
   .Scatter.GOOGL_Adj_full_stop_Close.II :Scatter   [Date]   (GOOGL Adj. Close)

:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)

:Overlay
   .Curve.I                        :Curve   [Date]   (Portfolio Total_ema)
   .Scatter.Portfolio_Total_ema.I  :Scatter   [Date]   (Portfolio Total_ema)
   .Scatter.Portfolio_Total_ema.II :Scatter   [Date]   (Portfolio Total_ema)

...

:Overlay
   .Curve.I                           :Curve   [Date]   (PG Adj. Close)
   .NdOverlay.I                       :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.PG_Adj_full_stop_Close.I  :Scatter   [Date]   (PG Adj. Close)
   .Scatter.PG_Adj_full_stop_Close.II :Scatter   [Date]   (PG Adj. Close)

:Overlay
   .Curve.I                           :Curve   [Date]   (PG Adj. Close)
   .NdOverlay.I                       :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.PG_Adj_full_stop_Close.I  :Scatter   [Date]   (PG Adj. Close)
   .Scatter.PG_Adj_full_stop_Close.II :Scatter   [Date]   (PG Adj. Close)

:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)

:Overlay
   .Curve.I                        :Curve   [Date]   (Portfolio Total_ema)
   .Scatter.Portfolio_Total_ema.I  :Scatter   [Date]   (Portfolio Total_ema)
   .Scatter.Portfolio_Total_ema.II :Scatter   [Date]   (Portfolio Total_ema)

:Overlay
   .Curve.I                           :Curve   [Date]   (Portfolio Total_bb_rsi)
   .Scatter.Portfolio_Total_bb_rsi.I  :Scatter   [Date]   (Portfolio Total_bb_rsi)
   .Scatter.Portfolio_Total_bb_rsi.II :Scatter   [Date]   (Portfolio Total_bb_rsi)